In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from yaml import safe_load

In [2]:
from StringProcessing import preprocessInput, encode_pad_tag, get_target_groups_from_connected_components

In [3]:
from SiameseModel import ContrastiveLoss, CosineDistance
from ModelEvaluation import find_Model_ranks

tf.version:  2.9.1
tf.keras.version:  2.9.0
tf.config.devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-10-24 11:04:36.947679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 11:04:36.973481: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 11:04:36.973749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 11:04:36.974799: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [4]:
with open('LSTM65.yaml', 'r') as file:
    model_config = safe_load(file)

In [5]:
model = tf.keras.models.load_model(
    './saved_models/{}'.format(model_config["Model_Name"]), 
    custom_objects = {
        'CosineDistance': CosineDistance,
        'ContrastiveLoss': ContrastiveLoss
    }, 
    compile=True, 
    options=None
)
model.summary()

2022-10-24 11:04:37.483846: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 11:04:37.484033: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 11:04:37.484142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 11:04:37.484281: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-24 11:04:37.484391: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

2022-10-24 11:04:43.949664: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-24 11:04:43.995764: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-24 11:04:44.004404: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-24 11:04:44.036074: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 5 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-10-24 11:04:44.140404: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 13 outputs but the _output_shapes at

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 65)]         0           []                               
                                                                                                  
 model (Functional)             (None, 65)           76425       ['input_2[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['model[0][0]',            

# Preprocess targets

In [6]:
target_sequences = encode_pad_tag(preprocessInput(filename="../data/NP_subset_LB_common_names.csv", maxlen= model_config['MAXLEN'], noise= None, balance=False, reflexive = False), match = 1, distance = 0, maxlen = model_config['MAXLEN'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   latin_binomial  70 non-null     object
 1   common_name     70 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['latin_binomial', 'common_name'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 0 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 

# Latin Binomial

In [7]:
LatinBinomials = target_sequences.iloc[:,[0,2]]
LatinBinomials.head(5)

,x,Processed_x
0,EUTERPE OLERACEA,"[5, 21, 20, 5, 18, 16, 5, 27, 15, 12, 5, 18, 1..."
1,ALOE VERA,"[1, 12, 15, 5, 27, 22, 5, 18, 1, 0, 0, 0, 0, 0..."
2,MALUS DOMESTICA,"[13, 1, 12, 21, 19, 27, 4, 15, 13, 5, 19, 20, ..."
3,MALUS PUMILA,"[13, 1, 12, 21, 19, 27, 16, 21, 13, 9, 12, 1, ..."
4,WITHANIA SOMNIFERA,"[23, 9, 20, 8, 1, 14, 9, 1, 27, 19, 15, 13, 14..."


# Common Names

In [8]:
CommonNames = target_sequences.iloc[:,[1,3]]
CommonNames.head(5)

,y,Processed_y
0,ACAI,"[1, 3, 1, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,ALOE VERA,"[1, 12, 15, 5, 27, 22, 5, 18, 1, 0, 0, 0, 0, 0..."
2,APPLE CIDER VINEGAR,"[1, 16, 16, 12, 5, 27, 3, 9, 4, 5, 18, 27, 22,..."
3,APPLE CIDER VINEGAR,"[1, 16, 16, 12, 5, 27, 3, 9, 4, 5, 18, 27, 22,..."
4,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ..."


## Find equivalence groups

In [9]:
equivalence_set = get_target_groups_from_connected_components(target_sequences)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   latin_binomial  70 non-null     object
 1   common_name     70 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['latin_binomial', 'common_name'], dtype='object')
	New names: Index(['x', 'y'], dtype='object')
Dropping row with empty cells:
	Dropped 0 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 0 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 2 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtyp

#  Preprocess unmapped sequences

In [11]:
unmapped_sequences = preprocessInput("../data/unmapped-faers-drug-strings-Q12004-through-Q2-2021-extended-NP-vocab-09012022.tsv", maxlen= 65, balance=False, reflexive = False, noise = None, sep="\t")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389386 entries, 0 to 389385
Data columns (total 1 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   drug_name_original  389385 non-null  object
dtypes: object(1)
memory usage: 3.0+ MB
None
Processing file: ----------------------------------------
Renaming colums:
	Current names: Index(['drug_name_original'], dtype='object')
	New names: Index(['x'], dtype='object')
Dropping row with empty cells:
	Dropped 1 rows with empty cells.
	Cleaning string sequences.
	Uppercasing string sequences.
Dropping sequences longer than the maxlen of 65:
	Dropped 7751 that exceeded the maximum sequence length.
Dropping duplicate sequences:
	Dropped 41849 duplicate sequences.
Done processing: ---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339785 entries, 0 to 339784
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  -------

In [12]:
unmapped_sequences = encode_pad_tag(unmapped_sequences, match = 1, distance = 0, maxlen = model_config['MAXLEN'])

Encoding and Padding: ----------------------------------
	Processing x
Tagging: -----------------------------------------------


# Subset we will consider for the experiment

In [13]:
# unmapped_sequences = unmapped_sequences["x"].transform(lambda x: x.split(" ")).explode()

In [14]:
# unmapped_sequences = encode_pad_tag(unmapped_sequences.to_frame(), match = 1, distance = 0, maxlen = model_config['MAXLEN'])

In [15]:
# unmapped_sequences["sequence_id"] = unmapped_sequences.index

In [16]:
# unmapped_sequences.reset_index()

In [17]:
# unmapped_sequences.loc[0, [unmapped_sequences.columns[0]]].iloc[0:10]

In [18]:
# find_Model_ranks(model, df = unmapped_sequences, ranks = 20, targets= LatinBinomials, equivalence_set = equivalence_set, )

In [19]:
# equivalence_set[equivalence_set['x'] == "LIPODERMA PATCH"]["x"].any()

In [20]:
# results["x"] = results["x"].apply(lambda x: x + " *Synonyms:* " + str(equivalence_set[equivalence_set['x'] == x]["y"].to_list()) if equivalence_set[equivalence_set['x'] == x]["x"].any() else x)

In [21]:
results = find_Model_ranks(model, df = CommonNames, ranks = 20, targets = unmapped_sequences, equivalence_set = equivalence_set)

2022-10-21 14:26:54.350649: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1750] (One-time warning): Not using XLA:CPU for cluster.

If you want XLA:CPU, do one of the following:

 - set the TF_XLA_FLAGS to include "--tf_xla_cpu_global_jit", or
 - set cpu_global_jit to true on this session's OptimizerOptions, or
 - use experimental_jit_scope, or
 - use tf.function(jit_compile=True).

To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a
proper command-line flag, not via TF_XLA_FLAGS).
2022-10-21 14:26:56.811895: I tensorflow/compiler/xla/service/service.cc:170] XLA service 0x55c1c9b2f350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-10-21 14:26:56.811914: I tensorflow/compiler/xla/service/service.cc:178]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 with Max-Q Design, Compute Capability 7.5
2022-10-21 14:26:56.815138: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:263] disabling MLIR cras

In [22]:
results = results.iloc[:, ~results.columns.isin(['Processed_x', 'Match', 'Distance'])]

In [23]:
results

,y,Processed_y,rank1,rank2,rank3,rank4,rank5,rank6,rank7,rank8,...,rank11,rank12,rank13,rank14,rank15,rank16,rank17,rank18,rank19,rank20
0,ACAI,"[1, 3, 1, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",ACAI,ACIA,ACACIA,ACAQI,ADACAI,AI,AAC,AC,...,ACHIA,ACICAL,ACASIA,ACITAC,ACADIA,CAZIQ,ATACA,CAZ,ABAC,CAC
1,ALOE VERA,"[1, 12, 15, 5, 27, 22, 5, 18, 1, 0, 0, 0, 0, 0...",ALOE VERA ALOE VERA,ALOEVERA EXTRA,ALOVEX,ALOE VERA GEL,ALOE VERA GEL ALOE VERA,ALOEVESTA,ALOE FEROX,ALLEAVE,...,ALLIEVE,AJOVE,ALEAVE,A SALVE,NACLALOE VERA,ALEVE MAX,ALLEVE,LEVERA,ALOVERT,NADOLOE
2,APPLE CIDER VINEGAR,"[1, 16, 16, 12, 5, 27, 3, 9, 4, 5, 18, 27, 22,...",PALPERIDONEINVEGA,PEPHIDRINEGUAIFEN LA,ASPERGILLUS NIGER VAR NIGER,AMP ENERGY DRINKS,MEPERIDINE MG IV,MEDPERIDINEPROMETHAZINE,BENZOYL PEROXIDEERYTHROMYCIN GEL,PROAIR EMERGENCY INHALER,...,MIEPERIDINEPROMETHAZINE,MYERS MOISTURIZING CREAM,EXCEDRINE MIGRANE,MALEAT DE ERGOMETRINA ZENTIVA,LUPRON DEPOT INJ MG,PERCOCET GENER,PERCOCET GENERIC MG,LACTATO DE RINGER,PSEUDOEPHEDRINE HYDROCHLORIDE FEXOFENADINE ALL...,PEPCID OVER THE COUNTER
3,APPLE CIDER VINEGAR,"[1, 16, 16, 12, 5, 27, 3, 9, 4, 5, 18, 27, 22,...",PALPERIDONEINVEGA,PEPHIDRINEGUAIFEN LA,ASPERGILLUS NIGER VAR NIGER,AMP ENERGY DRINKS,MEPERIDINE MG IV,MEDPERIDINEPROMETHAZINE,BENZOYL PEROXIDEERYTHROMYCIN GEL,PROAIR EMERGENCY INHALER,...,MIEPERIDINEPROMETHAZINE,MYERS MOISTURIZING CREAM,EXCEDRINE MIGRANE,MALEAT DE ERGOMETRINA ZENTIVA,LUPRON DEPOT INJ MG,PERCOCET GENER,PERCOCET GENERIC MG,LACTATO DE RINGER,PSEUDOEPHEDRINE HYDROCHLORIDE FEXOFENADINE ALL...,PEPCID OVER THE COUNTER
4,ASHWAGANDA,"[1, 19, 8, 23, 1, 7, 1, 14, 4, 1, 0, 0, 0, 0, ...",ASHWAGANDHA MAGNESIUM,ASHWAGHANDA AND MAGNESIUM,ASPEGIC ASGEGIC,TELMISARTANHYDROCHLOROTHIAZID STADA,AISIWAN GIMERACILOTERACILTEGAFUR,ASPARAGINASE UNSPECIFIED ASPARAGINASE,ASPARAGINASE UNSPECIFIEDASPARAGINASE,ARISTOCHOL KONZENTRAT,...,SHORTACTING BETA AGONIST SABA,STEROIDS NOS NO INGREDIENTSSUBSTAMCES,PASSIFLORA INCARNATA PASSIFLORA INCARNATA,WATSONS HYDROCODONEAPAP,SERTRALIN GENERA,ESTROGEN AND PROGESTIN,MAITAKE D FRACTION STANDARD,ASPARGINASE ASPARGINASE,WATSONACTIVIS GENERIC FOR CONCERTA,ASPARAGINIAN EXTRA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,VALERIAN,"[22, 1, 12, 5, 18, 9, 1, 14, 0, 0, 0, 0, 0, 0,...",VALERIAN,VALERIAAN,VALERIAN VALERIAN,VALERIANA,VALERIAAN VALERIAAN,VALERIANE RADIX,VALERIANA TEA,VALTRIAN,...,VALERIANA DISPERT,VALERIN JAPAN,VALERIANA COMPOSTA,VALERIANA EXTRACT,VALERIANA OFFICINALIS TABLETS,VALERIANA OFFICINALIS ROOT UNKNOWN FOR BALDRIAN,VALERIAN ROOT TEA,VALERIAN VALERIANA OFFICINALIS ROOT,VALERIN MAX,VALERIAN ROOT VALERIANAN OFFICINALIS ROOT
66,WHEAT GRASS,"[23, 8, 5, 1, 20, 27, 7, 18, 1, 19, 19, 0, 0, ...",WEST WHEAT GRASS,WHEAT,WHEAT GRASS DRINK,WHEATGRASS JUICE,WHEAT GRASS JUICE,WHITE CROSSES,TYLENOL ACHES STRAINS,WHEAT DEXTRAN,...,DIPHTHERIAACELLULAR PERTUSSIS TETANUS,WHEAT GERM,VARICELLAZOSTER PRESUMED VACCINE,WHEATDEXTRIN,VELIVET TRIPHASIC REGIMEN,FERRLECIT ASCORBIC ACID FERRIC SODIUM CITRATE ...,VIACTIV CALCIUM WD K,CITRUS CALCIUM PLUS D MAXIUM,VARIOUS CREAMS,N ACETLY L CYSTEINE
67,WOOD SPIDER,"[23, 15, 15, 4, 27, 19, 16, 9, 4, 5, 18, 0, 0,...",UNDISCLOSED MEDICATIONS TAKEN AS NEEDED,SOD POLYSTY SUSPENDED,NO DRUG SPECIFIED,DOLOSPEDIFEN,OPIOIDS UNSPECIFIED,HTD AGONIST UNSPECIFIED,GOODYS POWDER,CORTICOIDS UNSPECIFIED,...,OLIMEPIRIDE,ZONISISMIDE,MEDOSPIR,SOY ISOFLANOIDS,OPIOID UNSPECIFIED,DEPOLAN SWEDEN,ESMODENDSR,OPIOID NOT SPECIFIED,DICLOFENAC SODIUM SR,DICOFENAC SODIUM GEL
68,WOODLAND HAWTHORN,"[23, 15, 15, 4, 12, 1, 14, 4, 27, 8, 1, 23, 20...",DERMASOFT SORBOLENE CREAM WITH GLYCERIN,COMBINATIONS OF ANTIHYPERTENSIVES IN ATCGR,RECOMBINANT HUMAN GRANULOCYTE STIMULATING FACT...,DOSS COLOXYL WITH DANTHRON,DELSORATADINE RATIOPHARM,DEXTROMETHORPANPHENYLEPHRINE,CANDESARTANHYDROCHLOROTHIAZID GENERIC RATIOPHARM,WOMENS ONEADAY WITH IRON,...,OPALMON LIMAPROST ALFADEX TABLET UG,LEVODOPABENSERAZID RATIOPHARM,CONTRAST USED IN ANGIOPLASTY,LOSARTAN POTHYDROC

In [24]:
for i in results.index:
    results.loc[i].to_csv("./Novelty_Results/PT_{}.txt".format(i), sep='\n', header= False, index=False)

----------------------

In [25]:
results = find_Model_ranks(model, df = LatinBinomials, ranks = 20, targets = unmapped_sequences, equivalence_set = equivalence_set)

In [26]:
results = results.iloc[:, ~results.columns.isin(['Processed_x', 'Match', 'Distance'])]

In [27]:
results

,x,rank1,rank2,rank3,rank4,rank5,rank6,rank7,rank8,rank9,...,rank11,rank12,rank13,rank14,rank15,rank16,rank17,rank18,rank19,rank20
0,EUTERPE OLERACEA,EUTERPE OLERACEA,UVESTEROL ADEC,NEURAZEPAM,METROGEL ORACEA,UREPEARL CREAM,ENSURE CLEAR,NEUROEZE CREAM,ASACAL,TERESTEROL CREAM,...,UREA COLD CREAM,UNREPORTED MUSCLE RELAXER,EFEROX EFEKA,ACAI,ECONOZOLE CREAM,EXAZEPAM,EQUATE RESTORE PM,NERVE CREAM,BENAZERPREAL,EMERVEL TOUCH
1,ALOE VERA,ALOE VERA ALOE VERA,ALOEVERA EXTRA,ALOVEX,ALOE VERA GEL,ALOE VERA GEL ALOE VERA,ALOEVESTA,ALOE FEROX,ALLEAVE,ALOE VERAALOE VERA CREAM,...,ALLIEVE,AJOVE,ALEAVE,A SALVE,NACLALOE VERA,ALEVE MAX,ALLEVE,LEVERA,ALOVERT,NADOLOE
2,MALUS DOMESTICA,MUSCLE MEDICATION,MUSCLE SPASM MEDICATION,MALUS SPP VINEGAR EXTRACT,MUCOSTA MUCOSTA,DAE BULK ULMUS AMERICANA,MUSCLE RELAXER DEXA SOMETHING,MUSCLE RELAXER STARTS WITH C,MEDICAMENT EXPERIMENTAL,MAXIMUM STRENGTH OMEGA,...,MUSCLE IMPACT,SLEEP MEDICATION,MEGARED PLANT OMEGA,SLEEP MEDICAITON,MEIJER JR STRENGTH,ALKA SELTZER MORNING RELIEF EFFERVESCENT,DULERA MCGACT,MELPERONE MELPERONE MG,MAXIMUS AZ COM OMEGA,MUSCLE JOINT PAIN RELIEF RUB KAREWAY
3,MALUS PUMILA,SLEEPYTIME HERB,SLEEP DEVICE,SELEDIE IUML,MOUSE EPITHELIA,SLEEP AID PILL,BKOMPLEX LECIVA,MEASLES MUMPS AND RUBELLA,MULTILEX TM MINERALS,MALUS SPP,...,MALUS SYLVESTRIS DRY FRUIT,MLSE DEVLEES MLSC,MULTI MEGA MINERALS,ALBUMINA,PLEXUS DRINK,AMLODIPINE WRAMIPRIL,MUSCLE SPASM PILLS,SOLUMEDROL MG IVP,MULTIPLE NEW DRUGS,ALLUPURINAL
4,WITHANIA SOMNIFERA,ESTROGEN AND PROGESTIN,LESS STRONGER CLEANSER,ASHWAGHANDA AND MAGNESIUM,AISIWAN GIMERACILOTERACILTEGAFUR,SHISEIDO MOISTURIZER,ORGANIC NITRATES ORGANIC NITRATES,ASHWAGANDHA MAGNESIUM,ESTROGENS UNSPECIFIED AND PROGESTERONE,ESTROGEN UNSPECIFIED AND PROGESTERONE,...,STEROIDS NOS NO INGREDIENTSSUBSTAMCES,STEROIDS NOS NO INGREDIENTSSUBSTANCES,MIESSENCE ORGANIC PURIFYING MOISTURIZER,STRONGER AND NEO MINOPHAGEN C,SCREENING PHASE SCREENING PHASE,STRONGER NEOMINOPHAGEN CP,STRONGER NEO MINOPHAGEN C,EURODIN DIHYDROERGOCRISTINE MESILATE,STEROIDS NOSNO INGREDIENTSSUBSTANCES,SIBERIAN GINSENG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,VALERIANA OFFICINALIS,VALERIANA OFFICINALISVALERIANA OFFICINALIS,VALERIANA OFFICINALIS VALERIANA OFFICINALIS,VALERIANVALERIANA OFFICINALIS,VALERIAN VALERIANA OFFICINALIS,VALERIANA OFFICINALIS VALRIANA OFFICINALIS,VALERIANA OFFICIALS VALERIANA OFFICINALIS,VALERIN VALERIANA OFFICINALIS,VALERIAN ROOT VALERIANA OFFICINALIS,VALERIANA OFFICINALIS TABLETS,...,VALERIANA TEA,VALERIAN ROOT DRY EXTRACT VALERIANA OFFICINALS,BALDRIAN VALERIANA OFFICINALIS,VALERIAN ROOT,VALERIAN ROOT TEA,VALERIANAE COMP,VALERIAN VALERIAN,VALERIAN VALERIANA OFFICINALIS ROOT,VALERIAN ROOT VALERIANAN OFFICINALIS ROOT,VALERIANA OFFICINALIS ROOT UNKNOWN FOR BALDRIAN
66,TRITICUM AESTIVUM,ZEVALIN THERAPY RITUXIMAB INDIUMIBRITUMOMAB TI...,DAE BULK TRITICUM AESTIVUM,CITRUCEL WITH MAGNESIUM,YTARCIS YTTRIUM,YTRACIS YTTRIUM,WHEAT,PENICILLINUM CRYSTALLISATUM,IBRITUMOMAB TIUXETAN AND YTTRIUM,EVALIN THERAPYRITUXIMAB INDIUMIBRITUMOMAB TIUX...,...,ACTILAX FICUS CARICASENNOSIDE ABTRITICUM AESTIVUM,CITRICAL WITH MEGNESIUM,CITRICAL WITH MAGNESIUM,YTRACISYTTRIUM,PRIVATE LABEL NCHHICOTINE TRANSTHERAPEUTICSYSTEM,NATURE MULTI ADULT MAGNESIUM,TELEBRIX IOXITALAMATE SODIUM,IN ZEVALIN IBRITUMOMA TIUXETAN YTTRIUM,TAE BULK ACREMONIUM STRICTUM,TICILIMUMAB
67,HARPAGOPHYTUM PROCUMBENS,ANTIHYPERGLYCEMIC DRUGS,HYDROXYPROGEST CAPR SDV MGML AMNEAL PHARMACEUT...,ANTIHYPERGLYCEMICS,HYDROXYPROPYL CELLULOSE OPHTHALMIC INSERT,CHILDRENS TYLENOL PLUS COUGH MG MCNEIL,ANTIHYPERGLYCAEMICS,HEPARINS LOWMOLECULARMASS,HYD POLCPM SUS,DEVILS CLAW HARPAGOPHYTUM PROCUMBENS,...,HYPOGLYCEMIA DRUGS,HYLANDSS HOMEOPATHIC NATURAL HYLANDS,ANTIHYPERGLYCEMIC MEDICATIONS,HYDROCODACETAM MG MALLINCKRODT,HY POLCPM SUS,HYPOGLYCEMIC DRUGS,OHPC HYDROXY PROGESTERONE CAPROATE EMINENT SER...,HYD POL CPM SUS,HYLANDS HOEMOPATHIC HYLANDS INC,EQUATE CALCIUM PLUS D
68,CRATAEGUS LAEVIGATA,CETRABEN LIGHT LIQUID PARAFFINWHITE S

In [28]:
for i in results.index:
    results.loc[i].to_csv("./Novelty_Results/LB_{}.txt".format(i), sep='\n', header= False, index=False)

------------------------------------------------

In [ ]:
target_sequences.rename(columns={"x": "LB", "y": "PT"})

In [15]:
target_sequences.to_csv('./Novelty_Results/70_target_NPs_LB2PT.csv')

In [ ]:
# dUnique_df = pd.read_pickle("../data/dUnique_df.pkl")

In [30]:
# latin_2_common = preprocessInput('../data/vocabulary_LB_to_common_names_09082022.tsv', maxlen= model_config['MAXLEN'], reflexive=True, balance = False, sep='\t')

In [31]:
# common_2_latin = preprocessInput('../data/positive-unmapped-pairs-common-name-or-latin-binomial-copies.tsv', maxlen= model_config['MAXLEN'], reflexive=True, balance = False, sep='\t')

In [32]:
# gsrs_2_common = preprocessInput('../data/positive-unmapped-pairs-gsrs-name-to-common-name.tsv', maxlen= model_config['MAXLEN'], reflexive=True, balance = False, sep='\t')

In [33]:
# equivalence_set = pd.concat([latin_2_common, common_2_latin, gsrs_2_common], ignore_index=True)
# equivalence_set.drop_duplicates(subset=['x', 'y'], inplace=True)